In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
customers_df = spark.read \
    .format("parquet") \
        .load("abfss://bronze@salesstaradlsstorageacc.dfs.core.windows.net/customers")

In [0]:

customers_df.display()


In [0]:
customers_df.display()

In [0]:
customers_df = customers_df.withColumn("domain", split(col("email"), "@").getItem(1))

In [0]:
domain_count_df = customers_df.groupBy("domain").count()

In [0]:
domain_count_df.display() 


In [0]:
gmail_df  = customers_df.filter(col("domain")=="gmail.com")

In [0]:
customers_df.display()

In [0]:
hotmail_df = customers_df.filter(col("domain")=="hotmail.com")
hotmail_df.display()


yahoo_df = customers_df.filter(col("domain")=="yahoo.com")
yahoo_df.display()


In [0]:
customers_df = customers_df.withColumn("Full_name",concat(col("first_name"),lit(" "),col("last_name")))

customers_df.display()

In [0]:
customers_df.drop("_rescued_data").display()

In [0]:
customers_df.write.mode("append") \
    .format("delta") \
        .save("abfss://silver@salesstaradlsstorageacc.dfs.core.windows.net/customers")

In [0]:
dbutils.widgets.text("catalog_name", "")

In [0]:
catalog_name = dbutils.widgets.get("catalog_name")

In [0]:
spark.sql(f""" 
create table if not exists {catalog_name}.silver.customer_silver
using delta 
location "abfss://silver@salesstaradlsstorageacc.dfs.core.windows.net/customers"
""")